#### Problem Statement:

A US bike-sharing provider `BoomBikes` has a daily dataset on the rental bikes based on various environmental and seasonal settings. It wishes to use this data to understand the factors affecting the demand for these shared bikes in the American market and come up with a mindful business plan to be able to accelerate its revenue as soon as the ongoing lockdown due to corona pandemic comes to an end.

**the company wants to know**:


- Which variables are significant in predicting the demand for shared bikes.


- How well those variables describe the bike demands


This solution is divided into the following main sections: 
- Data understanding/exploration
- Data Visualisation 
- Data preparation/cleaning
- Model building and evaluation
- Residual Analysis (verifying the assumptions of Linear Model)


### 1. Data Understanding/Exploration

Let's first import the required libraries and have a look at the dataset and understand the size, attribute names etc.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Reading the dataset
bike = pd.read_csv("day.csv")

In [ ]:
# Let's take a look at the first few rows
bike.head(3).append(bike.tail(3))

In [ ]:
# Let's look at the number of rows and columns in the dataset
bike.shape

In [ ]:
# Understanding the feature names in the dataset
print(bike.columns)
bike.dtypes.value_counts()

In [ ]:
# Summary of the dataset: 730 rows, 16 columns, no null values
print(bike.info())

In [ ]:
# Getting insights of the features
bike.describe()

#### Understanding the Data Dictionary and parts of Data Preparation

The data dictionary contains the meaning of various attributes; some of which are explored and manipulated here:

In [ ]:
# Assigning string values to different seasons instead of numeric values. These numeric values may misindicate some order to it.
print(bike['season'].value_counts())
# 1=spring
bike.loc[(bike['season'] == 1) , 'season'] = 'spring'

# 2=summer
bike.loc[(bike['season'] == 2) , 'season'] = 'summer'

# 3=fall
bike.loc[(bike['season'] == 3) , 'season'] = 'fall'

# 4=winter
bike.loc[(bike['season'] == 4) , 'season'] = 'winter'

In [ ]:
# Checking whether the conversion is done properly or not and getting data count on the basis of season
bike['season'].astype('category').value_counts()

In [ ]:
# year (0: 2018, 1:2019)
bike['yr'].astype('category').value_counts()

In [ ]:
# Assigning string values to different months instead of numeric values which may misindicate some order to it.
# A function has been created to map the actual numbers to categorical levels.
# Check the data before change
print(bike['mnth'].astype('category').value_counts())
def object_map(x):
    return x.map({1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul',8: 'Aug',9: 'Sept',10: 'Oct',11: 'Nov',12: 'Dec'})

# Applying the function to the two columns
bike[['mnth']] = bike[['mnth']].apply(object_map)

In [ ]:
# Checking whether the conversion is done properly or not and getting data count on the basis of month
bike['mnth'].astype('category').value_counts(ascending = True)

In [ ]:
# whether day is a holiday or not (0: No, 1: Yes)
bike['holiday'].astype('category').value_counts()

In [ ]:
# Assigning string values to weekdays instead of numeric values. These values may misindicate some order to it.
# A function has been created to map the actual numbers to categorical levels.
def str_map(x):
    return x.map({1: 'Wed', 2: 'Thur', 3: 'Fri', 4: 'Sat', 5: 'Sun', 6: 'Mon', 0: 'Tue'})

# Applying the function to the two columns
bike[['weekday']] = bike[['weekday']].apply(str_map)

In [ ]:
# Checking whether the conversion is done properly or not and getting data count on the basis of weekdays
bike['weekday'].astype('category').value_counts()

In [ ]:
# if a day is neither weekend nor a holiday it takes the value 1, otherwise 0
bike['workingday'].astype('category').value_counts()

In [ ]:
# Optional: Replacing long weathersit names into string values for better readability and understanding

# 1-Clear, Few clouds, Partly cloudy, Partly cloudy
bike.loc[(bike['weathersit'] == 1) , 'weathersit'] = 'A'

# 2-Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
bike.loc[(bike['weathersit'] == 2) , 'weathersit'] = 'B'

# 3-Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
bike.loc[(bike['weathersit'] == 3) , 'weathersit'] = 'C'

# 4-Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
bike.loc[(bike['weathersit'] == 4) , 'weathersit'] = 'D'

In [ ]:
# Extracting the type of weather situations present in the data
print(bike['weathersit'].unique())
print(bike['weathersit'].astype('category').value_counts(ascending = True))

In [ ]:
# Converting date to datetime format
print("As is:",bike['dteday'].dtypes)
bike['dteday']=bike['dteday'].astype('datetime64')
print("Now:", bike['dteday'].dtypes)

### Conclusion: 1. Data Understanding/Exploration

In this section, we have analysed the given dataset w.r.to it's structure. In the process, we have changed the data types of few columns and also changed some of the values based on Data Dictionary.

### 2. Data Visualisation

Let's now spend some time doing what is arguably the most important step - **Data Content Analysis**.
- Understanding the distribution of various numeric variables 
- If there is some obvious multicollinearity going on, this is the first place to catch it
- Here's where you'll also identify if some predictors directly have a strong association with the outcome variable

We'll visualise our data using `matplotlib` and `seaborn`.

In [ ]:
# temp: temperature in Celsius
sns.set_style("darkgrid")
sns.distplot(bike['temp'],bins = 15, color = 'b').set(title='Temperature in Celsius')
plt.show()

In [ ]:
# feeling temperature
sns.distplot(bike['atemp']).set(title='Feeling Temperature')
plt.show()

In [ ]:
# humidity
sns.distplot(bike['hum'], color = 'g').set(title='Humidity')
plt.show()

In [ ]:
# wind speed
sns.distplot(bike['windspeed'], color = 'r').set(title='Wind Speed')
plt.show()

In [ ]:
# Target variable: count of total rental bikes including both casual and registered
sns.distplot(bike['cnt'], bins = 10).set(title='Count of Rented Bikes')
plt.show()

In [ ]:
# 0: 2018, 1: 2019
sns.barplot(x='yr', y='cnt', data=bike)
plt.show()

In [ ]:
sns.barplot(x='weekday', y='cnt', data=bike)
plt.show()

In [ ]:
# Holiday - 0: No, 1: Yes
sns.barplot(x='holiday', y='cnt', data=bike)
plt.show()

In [ ]:
# weathersit
# 'A'- Clear, Few clouds, Partly cloudy, Partly cloudy
# 'B'- Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
# 'C'- Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
# 'D'- Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

sns.barplot(x='weathersit', y='cnt', data=bike)
plt.show()

In [ ]:
# All categorical variables in the dataset
bike_categorical = bike.select_dtypes(exclude=['float64','datetime64','int64'])
print("Only categorical varibales:", bike_categorical.columns)

In [ ]:
bike_categorical.head(3).append(bike_categorical.tail(3))

In [ ]:
bike.dtypes.value_counts()

#### Visualising Categorical Variables

As you might have noticed, there are a few categorical variables as well. Let's make a boxplot for some of these variables.

In [ ]:
plt.figure(figsize=(20, 20))  
plt.subplot(3,3,1)
sns.boxplot(x = 'season', y = 'cnt', data = bike)
plt.subplot(3,3,2)
sns.boxplot(x = 'holiday', y = 'cnt', data = bike)
plt.subplot(3,3,3)
sns.boxplot(x = 'workingday', y = 'cnt', data = bike)
plt.subplot(3,3,4)
sns.boxplot(x = 'weathersit', y = 'cnt', data = bike)
plt.subplot(3,3,5)
sns.boxplot(x = 'mnth', y = 'cnt', data = bike)
plt.subplot(3,3,6)
sns.boxplot(x = 'weekday', y = 'cnt', data = bike)
plt.subplot(3,3,7)
sns.boxplot(x = 'yr', y = 'cnt', data = bike)
plt.show()

#### Visualising Numeric Variables

Let's make a pairplot of all the numeric variables

In [ ]:
# Converting "casual","registered" and "cnt" Integer/discreate variables to float. 
# This step is performed to seperate out categorical variables like 'yr','holiday','workingday' which have binary values in them
Int_Var_List = ["casual","registered","cnt"]

for var in Int_Var_List:
    bike[var] = bike[var].astype("float")
bike[Int_Var_List].head(3)

In [ ]:
# All numeric variables in the dataset
bike_numeric = bike.select_dtypes(include=['float64'])
bike_numeric.head()

We can better plot correlation matrix between variables to know the exact values of correlation between them. Also, a heatmap is pretty useful to visualise multiple correlations in one plot.

In [ ]:
# Correlation matrix
cor = bike_numeric.corr()
cor

In [ ]:
#removing atemp as it is highly correlated with temp
bike.drop('atemp',axis=1,inplace=True)    

In [ ]:
print(bike.shape)
print(bike.dtypes.value_counts())

### Conclusion: 2. Data Visualization

In this section, we have looked at the actual data content.We plotted few distribution plots and a correleation Matrix and heatmap for numerical varibales. We also removed one of the features "atemp" as it was highly correlated with the another varibales "temp". 

## 3. Data Preparation 


#### Data Preparation

Let's now prepare the data and build the model.
Note that we had not included 'yr', 'mnth', 'holiday', 'weekday' and 'workingday' as object variables in the initial data exploration steps so as to avoid too many dummy variables creation. They have binary values: 0s and 1s in them which have specific meanings associated with them.

In [ ]:
# Subset all categorical variables
bike_categorical=bike.select_dtypes(include=['object'])
bike_categorical.head()

#### Dummy Variables
The variable `season`,`mnth`,`weekday` and `weathersit` have different levels. We need to convert these levels into integers. 

For this, we will use something called `dummy variables`.

In [ ]:
# Convert into dummies
bike_dummies = pd.get_dummies(bike_categorical, drop_first=True)
bike_dummies.head()

In [ ]:
# Drop categorical variable columns
bike = bike.drop(list(bike_categorical.columns), axis=1)

In [ ]:
# Concatenate dummy variables with the original dataframe
bike = pd.concat([bike, bike_dummies], axis=1)

In [ ]:
# Let's check the first few rows
bike.head()

In [ ]:
# Check Column 'instant'[Record Index] and 'dteday'[Date]

print(len(bike.instant.unique()))
print(len(bike.dteday.unique()))

In [ ]:
# Drop the 'instant' and 'dteday' column as they of not any use to us for the analysis
bike=bike.drop(['instant','dteday'], axis = 1, inplace = False)
bike.head(3).append(bike.tail(3))

### Conclusion: 3. Data Preparation

The main task done in this section are:
- create the respective dummy variables
- delete the irrelevant data

## 4. Model Building and Evaluation

Let's start building the model. The first step to model building is the usual test-train split. So let's perform that

In [ ]:
# Before splitting, make a copy of the cleaned data frame
bike_c = bike.copy()
print(bike.shape)
print(bike_c.shape)

In [ ]:
# Split the dataframe into train and test sets
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(bike, train_size=0.7, test_size=0.3, random_state=100)

In [ ]:
print(df_train.shape)
df_train.head(10)

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
# from sklearn.preprocessing import StandardScaler - in case you want to use Standardization method

In [ ]:
scaler = MinMaxScaler()

In [ ]:
bike_numeric.columns  # created in previous section

In [ ]:
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables
var = ['temp', 'hum', 'windspeed','casual','registered','cnt']

df_train[var] = scaler.fit_transform(df_train[var])

In [ ]:
df_train[var].describe()

# Please note that min is 0 and max. is 1 now

As expected, the variables have been appropriately scaled.

In [ ]:
df_train.describe()

In [ ]:
# Let's check the correlation coefficients to see which variables are highly correlated
plt.figure(figsize = (30, 30))
sns.heatmap(df_train.corr(), annot = True, cmap="YlGnBu")
plt.show()

As you might have noticed, `temp` seems to the correlated to `cnt` the most, after 'casual' and 'registered'. Let's see a pairplot for `temp` vs `cnt`.

In [ ]:
plt.figure(figsize=(20, 20)) 

plt.subplot(6,6,1)
plt.scatter(df_train.temp, df_train.cnt)

plt.subplot(6,6,2)
plt.scatter(df_train.casual, df_train.cnt)

plt.subplot(6,6,3)
plt.scatter(df_train.registered, df_train.cnt)
plt.show()

#### Dividing into X and Y sets for the model building

In [ ]:
# Dropping 'casual' and 'registered' as together they add up to cnt
y_train = df_train.pop('cnt')
X_train = df_train.drop(["casual","registered"],axis=1) 

In [ ]:
print(X_train.shape)
X_train.head()

In [ ]:
y_train.shape

### Building the first model with all the features

Let's now build our first model with all the features.

In [ ]:
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train)
lr_model1 = sm.OLS(y_train, X_train_lm).fit()

lr_model1.params

In [ ]:
# Creating a model using sklearn Linear Regression
lm = LinearRegression()

# Fit a line
lm.fit(X_train, y_train)

In [ ]:
# Print the coefficients and intercept
print(lm.coef_)
print(lm.intercept_)

In [ ]:
# getting the model summary from statsmodel
lr_model1.summary()

This model has an Adjusted R-squared value of **84.5%** which seems pretty good. But let's see if we can reduce the number of features and exclude those which are not much relevant in explaining the target variable. 

#### Model Building Using RFE

Now, you have close to 28 features. It is obviously not recommended to manually eliminate these features. So let's now build a model using recursive feature elimination to select features. We'll first start off with an arbitrary number of features (15 seems to be a good number to begin with), and then use the `statsmodels` library to build models using the shortlisted features (this is also because `SKLearn` doesn't have `Adjusted R-squared` that `statsmodels` has).

In [ ]:
# Import RFE
from sklearn.feature_selection import RFE

# RFE with 15 features
lm = LinearRegression()
rfe1 = RFE(lm, 15)

# Fit with 15 features
rfe1.fit(X_train, y_train)

# Print the boolean results
print(rfe1.support_)           
print(rfe1.ranking_)  

#### Model Building and Evaluation 

Let's now check the summary of this model using `statsmodels`.

In [ ]:
# Import statsmodels
import statsmodels.api as sm  

# Subset the features selected by rfe1
col1 = X_train.columns[rfe1.support_]

# Subsetting training data for 15 selected columns
X_train_rfe1 = X_train[col1]

# Add a constant to the model
X_train_rfe1 = sm.add_constant(X_train_rfe1)
X_train_rfe1.head()

In [ ]:
# Fitting the model with 15 variables
lm1 = sm.OLS(y_train, X_train_rfe1).fit()   
print(lm1.summary())

Note that the new model built on the selected features doesn't show much dip in the accuracy in comparison to the model which was built on all the features. It has gone from **84.5%** to **84.4%**. This is indeed a good indication to proceed with these selected features.

But let's check for the multicollinearity among these variables.

In [ ]:
# Check for the VIF values of the feature variables. 
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
a=X_train_rfe1.drop('const',axis=1)

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs except for the constant
vif = pd.DataFrame()
vif['Features'] = a.columns
vif['VIF'] = [variance_inflation_factor(a.values, i) for i in range(a.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# RFE with 8 features
lm = LinearRegression()
rfe2 = RFE(lm, 8)

# Fit with 7 features
rfe2.fit(X_train, y_train)

# Print the boolean results
print(rfe2.support_)           
print(rfe2.ranking_)  

In [ ]:
# Import statsmodels
import statsmodels.api as sm  

# Subset the features selected by rfe1
col1 = X_train.columns[rfe2.support_]

# Subsetting training data for 7 selected columns
X_train_rfe2 = X_train[col1]

# Add a constant to the model
X_train_rfe2 = sm.add_constant(X_train_rfe2)
X_train_rfe2.head()

In [ ]:
# Fitting the model with 10 variables
lm2 = sm.OLS(y_train, X_train_rfe2).fit()   
print(lm2.summary())

Now let's check the VIF for these selected features and decide further.

In [ ]:
b=X_train_rfe2.drop('const',axis=1)

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs except for the constant
vif = pd.DataFrame()
vif['Features'] = b.columns
vif['VIF'] = [variance_inflation_factor(b.values, i) for i in range(b.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

From the model summary above, all the variables have p-value < 0.05 and from the p-value perspective, all variables seem significant. But notice that there are a few variables which have VIF > 5. We need to deal with these variables carefully.

So let's try removing 'hum' first having the maximum VIF and then check for it again. Dropping this variable may result in a change in other VIFs which are high.

In [ ]:
# Let's drop the 'hum' column
X_train_rfe2.drop("hum",axis=1,inplace=True)
X_train_rfe2

In [ ]:
X_train_rfe2 = sm.add_constant(X_train_rfe2)

# Now that we have removed one variable, let's fit the model with 6 variables
lm3 = sm.OLS(y_train, X_train_rfe2).fit()   
print(lm3.summary())

The model seems to be doing a good job. Let's also quickly take a look at the VIF values.

In [ ]:
c=X_train_rfe2.drop('const',axis=1)

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs except for the constant
vif = pd.DataFrame()
vif['Features'] = c.columns
vif['VIF'] = [variance_inflation_factor(c.values, i) for i in range(c.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# Making Prediction on train data set
y_train_cnt = lm3.predict(X_train_rfe2)

In [ ]:
y_train_cnt.describe()

In [ ]:
# Check the Mean of Error terms
residuals = y_train - y_train_cnt
mean_residuals = np.mean(residuals)
print("Mean of Residuals {}".format(mean_residuals))
residuals.describe()


We need to check if the residuals or error terms are normally distributed or not.

In [ ]:
# Plot the histogram of the error or residuals terms terms
fig = plt.figure()
sns.distplot((y_train - y_train_cnt), bins = 20, color = 'r')
# Plot heading
fig.suptitle('Error Terms', fontsize = 20)    
# Give the X-label
plt.xlabel('Errors', fontsize = 18)                         

In [ ]:
plt.figure(figsize=(15,5))
p = sns.scatterplot(y_train_cnt,residuals, color = "r")
plt.xlabel('y_pred', color = 'r')
plt.ylabel('Residuals', color = 'b')
plt.ylim(-1,1)
plt.xlim(0,1)
p = sns.lineplot([0,1],[0,0],color='g')
p = plt.title('Homoscedasticity Check')
plt.show()

**Error terms are independent of each other**

That means there should not be any auto-correlation between error terms.

In [ ]:
plt.figure(figsize=(15,5))
p = sns.lineplot(y_train_cnt,residuals,marker='o',color='purple')
plt.xlabel('y_pred')
plt.ylabel('Residuals')
plt.ylim(-1,1)
plt.xlim(0,1)
p = sns.lineplot([0,1],[0,0],color='green')
p = plt.title('Residuals vs fitted values plot for autocorrelation check')

### Making Predictions

We would first need to scale the test set as well. So let's start with that.

In [ ]:
X_train_rfe2

In [ ]:
# let's recall the set of variables which are to be scaled
var

In [ ]:
# df_test holds the test dataset for us
df_test[var] = scaler.transform(df_test[var])

In [ ]:
# Split the 'df_test' set into X and y after scaling
y_test = df_test.pop('cnt')
X_test = df_test.drop(["casual","registered"],axis=1)

In [ ]:
X_test.head()

In [ ]:
# Let's check the list 'col2' which had the 7 variables RFE had selected
col2=c.columns
print(len(col2))
col2

In [ ]:
# Let's subset these columns and create a new dataframe 'X_test_rfe1'
X_test_rfe2 = X_test[col2]

In [ ]:
# Add a constant to the test set created
X_test_rfe2 = sm.add_constant(X_test_rfe2)
X_test_rfe2.info()

In [ ]:
# Making predictions using our final model: lm3
y_pred = lm3.predict(X_test_rfe2)

In [ ]:
# Plotting y_test and y_pred to understand the spread

fig = plt.figure()
#plt.scatter(y_test, y_pred)
sns.regplot(x=y_test,y=y_pred,ci=None,color ='purple');
fig.suptitle('y_test vs y_pred', fontsize = 16, color = 'green')              # Plot heading 
plt.xlabel('y_test', fontsize = 16, color = 'blue')                          # X-label
plt.ylabel('y_pred', fontsize = 16, color = 'blue')                          # Y-label

From the above plot, it's evident that the model is doing well on the test set as well. Let's also check the R-squared and more importantly, the adjusted R-squared value for the test set.

In [ ]:
# r2_score for 8 variables on test dataset and it's predcition
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, y_pred)
print("r2 score is: ",r2_score)
n = len(X_test)
p = 7
print("and Adjusted r2 score is: ",1-(1-r2_score)*(n-1)/(n-p-1))


Thus, for the model with 7 variables, the r-squared on training and test data is about 79.6% and 78.35% respectively. The **Adjusted r-squared** on the train and test set is 79.3% and 77.63% respectively.

#### Checking the correlations between the final predictor variables

In [ ]:
# Figure size
plt.figure(figsize=(8,6))

# Heatmap
sns.heatmap(bike[col2].corr(), cmap="YlGnBu", annot=True)
plt.show()

This is the simplest model that we could build. The final predictors seem to have fairly low correlations. 

Thus, the final model consists of the **7 variables** mentioned above.One can go ahead with this model and use it for predicting count of daily bike rentals.



In [ ]:
print(type(lm3.params))
print(abs(lm3.params).sort_values(ascending = False)) # It's magnitude of the coefficinets that matters

#### Suggestion to BoomBikes Management

We can conclude from our model that below three features are the most influential features for BIke Rentals:
- Temperature : with coefficient `0.42`
- Weather C [3-Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds ]:  with coefficient `0.24`
- Year: with coefficient `0.23`

### Conclusion: 4. Model Building and Evaluation

In this section, we build our first model considering all the variables and then we made an informed choice based on VIF and p-values and finally made our baseline model with 7 variables. We also plotted the error term to check if they are normally distributed. Our regression line also looks pretty clustered around the central line.